In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np
import threading
from playsound import playsound
from gtts import gTTS

# Generate and save audio feedback
tts_up = gTTS("Up", lang="en")
tts_down = gTTS("Down", lang="en")
tts_up.save("up_new.mp3")
tts_down.save("down_new.mp3")

# Function to calculate angles using 2D coordinates
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ab, bc = a - b, c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0)) * 180.0 / np.pi
    return angle

# Function to play sound asynchronously
def play_sound(sound_file):
    threading.Thread(target=lambda: playsound(sound_file), daemon=True).start()

# Initialize MediaPipe Pose
mp_drawing, mp_pose = mp.solutions.drawing_utils, mp.solutions.pose

# Open camera
cap = cv2.VideoCapture(0)
counter, stage = 0, None
use_right_hand = True  # Default to right hand
completed_rep = False

# Start pose detection
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark

                # Extract coordinates based on the selected hand
                if use_right_hand:
                    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                else:
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

                # Calculate the elbow angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                # Convert coordinates for display
                shoulder_coords = tuple(np.multiply(shoulder, [640, 480]).astype(int))
                elbow_coords = tuple(np.multiply(elbow, [640, 480]).astype(int))
                wrist_coords = tuple(np.multiply(wrist, [640, 480]).astype(int))

                # Draw circles on landmarks
                cv2.circle(image, shoulder_coords, 5, (0, 255, 0), -1)  # Shoulder
                cv2.circle(image, elbow_coords, 5, (255, 0, 0), -1)  # Elbow
                cv2.circle(image, wrist_coords, 5, (0, 0, 255), -1)  # Wrist

                # Draw lines connecting the shoulder, elbow, and wrist
                cv2.line(image, shoulder_coords, elbow_coords, (0, 255, 255), 2)  # Shoulder to Elbow
                cv2.line(image, elbow_coords, wrist_coords, (0, 255, 255), 2)  # Elbow to Wrist

                # Display angle on screen
                cv2.putText(image, f'{int(angle)}°', elbow_coords,
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)

                # Progress bar (visual indicator of movement)
                bar_height = np.interp(angle, (50, 150), (100, 400))
                cv2.rectangle(image, (20, 100), (45, 400), (255, 255, 255), 2)
                cv2.rectangle(image, (20, int(bar_height)), (45, 400), (0, 255, 0), -1)
 
                # Curl counting logic
                if angle > 150 and stage != "up":
                    stage = "up"
                    play_sound("up_new.mp3")

                if angle < 50 and stage == "up":
                    stage = "down"
                    completed_rep = True
                    play_sound("down_new.mp3")
                
                # Hand switching logic
                if completed_rep and angle > 150:
                    counter += 1
                    completed_rep = False
                    use_right_hand = not use_right_hand  # Automatically switch hand
                    stage = None  # Reset stage

            # Display count, stage, and active hand
            cv2.putText(image, f'Count: {counter}', (480, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Stage: {stage}', (480, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Hand: {"Right" if use_right_hand else "Left"}', (480, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)

        except Exception as e:
            print(f"Error: {e}")

        # Show the output
        cv2.imshow('Virtual Gym Trainer - Single Hand Bicep Curl', image)

        # Exit on 'q' press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Clean up audio files
for file in ["up_new.mp3", "down_new.mp3"]:
    if os.path.exists(file):
        os.remove(file)



    Error 263 for command:
        open down_new.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close down_new.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: down_new.mp3
Exception in thread Thread-6 (<lambda>):
Traceback (most recent call last):
  File "C:\Users\Ravi\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Ravi\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Ravi\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ravi\AppData\Local\Temp\ipykernel_13880\554471343.py", line 26, in <lambda>
  File "C:\Users\Ravi\anaconda3\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "C:\Users\Ravi\anaconda3\Lib\site-packages\playsou